In [1]:
import xarray as xr
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

%load_ext autoreload
%autoreload 2

### Paths input data

In [2]:
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, RESULTS_DIR

# PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
# PATH_DT = OUTPUT_DATA_DIR / '/dT_data_rcmip_models.nc'
PATH_DT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


#### Uncertainty data from Chris

In [3]:
PATH_DT_UNCERTAINTY = OUTPUT_DATA_DIR / 'dT_uncertainty_data_FaIR_chris_ed02-3.nc'

## Set values:

In [4]:
first_y = 1750
last_y = 2100

**Set reference year for temperature change:**

In [5]:
ref_year = 2020

In [6]:
FIGURE_DIR = RESULTS_DIR / 'figures_recommendation/'

TABLE_DIR = RESULTS_DIR / 'tables_recommendation/'

In [7]:
from pathlib import Path
Path(FIGURE_DIR).mkdir(parents=True, exist_ok=True)
Path(TABLE_DIR).mkdir(parents=True, exist_ok=True)

In [8]:
percentile = 'percentile'
climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
name_deltaT = 'Delta T'


### Define variables to look at:

In [9]:
# variables to plot:
variables_erf_comp = [
    'ch4',
    'aerosol-total-with_bc-snow',
    #'aerosol-radiation_interactions',
    #'aerosol-cloud_interactions',
    'aerosol-total',
    'o3',
    'HFCs',
    # 'F-Gases|HFC',
    'bc_on_snow',
    'total_anthropogenic',
    #'total',
]
variables_in_sum = [
    'aerosol-total-with_bc-snow',
    'ch4',
    # 'aerosol-radiation_interactions',
    # 'aerosol-cloud_interactions',
    #'aerosol-total',
    'o3',
    'HFCs',
    #'bc_on_snow'
]

# total ERFs for anthropogenic and total:
#variables_erf_tot = ['total_anthropogenic',
#                     'total']

scenarios_fl_370 = ['ssp370', 'ssp370-lowNTCF-aerchemmip', 'ssp370-lowNTCF-gidden'  # Due to mistake here
                    ]

### Scenarios:

In [10]:
scenarios_fl = ['ssp119',
                'ssp126',
                'ssp245',
                'ssp370',
                'ssp370-lowNTCF-aerchemmip',
                #'ssp370-lowNTCF-gidden',
                'ssp370-lowNTCF-gidden',
                'ssp585'
               ]

In [11]:
median = 'median'
perc5 = '5th percentile'
perc95 = '95th percentile'
recommendation = 'recommendation'

## Open dataset:

### Integrate:
The code below opens the file generated in [2_compute_delta_T.ipynb](2_compute_delta_T.ipynb) by integrating

\begin{align*} 
\Delta T (t) &= \int_0^t ERF(t') IRF(t-t') dt' \\
\end{align*}

where IRF is the impulse response function and ERF is the effective radiative forcing from RCMIP. 

In [12]:
ds_DT = xr.open_dataset(PATH_DT)
ds_uncertainty = xr.open_dataset(PATH_DT_UNCERTAINTY)

In [13]:
for var in variables_erf_comp:
    da5 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p05-p50']    
    da95 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p95-p50']
    da5.plot(label=var)
    da95.plot(label=var)

In [14]:
ds_uncertainty.variable

<xarray.DataArray 'variable' (variable: 8)>
array(['HFCs', 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)', 'aerosol-total',
       'bc_on_snow', 'ch4', 'o3', 'total_anthropogenic',
       'aerosol-total-with_bc-snow'], dtype=object)
Coordinates:
  * variable  (variable) object 'HFCs' ... 'aerosol-total-with_bc-snow'

In [16]:
v_sum = 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)'
ds_uncertainty['p05-p50'].sel(year=2040, scenario='ssp119', variable=v_sum)#.variables#.plot()#, variable='%%SVG')%%SVG

<xarray.DataArray 'p05-p50' (base_period: 1)>
array([-0.126748])
Coordinates:
    scenario     <U6 'ssp119'
  * base_period  (base_period) int64 2020
    year         int64 2040
    variable     <U41 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)'

In [17]:

ds_uncertainty.variable

<xarray.DataArray 'variable' (variable: 8)>
array(['HFCs', 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)', 'aerosol-total',
       'bc_on_snow', 'ch4', 'o3', 'total_anthropogenic',
       'aerosol-total-with_bc-snow'], dtype=object)
Coordinates:
  * variable  (variable) object 'HFCs' ... 'aerosol-total-with_bc-snow'

In [18]:
ds_DT

<xarray.Dataset>
Dimensions:     (percentile: 1, scenario: 11, variable: 19, year: 351)
Coordinates:
  * year        (year) int64 1750 1751 1752 1753 1754 ... 2097 2098 2099 2100
  * variable    (variable) object 'co2' 'ch4' ... 'HFCs'
  * scenario    (scenario) object 'ssp534-over' ... 'ssp370-lowNTCF-gidden'
  * percentile  (percentile) object 'recommendation'
Data variables:
    ERF         (scenario, variable, year) float64 ...
    time        (year) datetime64[ns] ...
    delta_t     (year) float64 ...
    Delta T     (percentile, scenario, variable, year) float64 ...

## Merge uncertainty and original: 


In [19]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

<IPython.core.display.Javascript object>

hallo


### Add sum

In [20]:
_str = ''
_vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_in_sum]
for var in _vl: 
    _str += f'{var}, '

# ax.set_title('Temperature change, sum SLCF  (%s)' % _str[:-2])


vn_sum = 'Sum SLCF (%s)' % _str[:-2]
print(vn_sum)
#_st = vn_sum.replace('(','').replace(')','').replace(' ','_').replace(',','')+'.csv'

_da_sum  = ds_DT[name_deltaT].sel(variable=variables_in_sum).sum(variable)
#_da = ds_DT[name_deltaT].sel(variable=variables_erf_comp).sum(variable).sel(year=slice(int(s_y2), int(e_y2))) - ds_DT_sy
_da_sum#.assin_coord()
#_ds_check = ds_DT.copy()
ds_DT
#xr.concat([_ds_check[name_deltaT],_da_sum], dim=variable)

dd1=_da_sum.expand_dims(
    {'variable':
    [vn_sum]})
#dd1=dd1.to_dataset()

ds_DT = xr.merge([ds_DT,dd1])

Sum SLCF (Aerosols, Methane, Ozone, HFCs)


In [21]:
percentiles_to_keep = ['p05-p50','p16-p50','p84-p50','p95-p50']

In [22]:
_da = ds_uncertainty[percentiles_to_keep].to_array('percentile')
_ds = _da.rename(name_deltaT).to_dataset()

In [23]:
ds_DT= xr.concat([ds_DT[name_deltaT],_ds[name_deltaT]], dim='percentile').to_dataset()

In [24]:
ds_DT

<xarray.Dataset>
Dimensions:      (base_period: 1, percentile: 5, scenario: 11, variable: 20, year: 351)
Coordinates:
  * year         (year) int64 1750 1751 1752 1753 1754 ... 2097 2098 2099 2100
  * variable     (variable) object 'HFCs' ... 'volcanic'
  * scenario     (scenario) object 'ssp119' 'ssp126' ... 'ssp534-over' 'ssp585'
  * base_period  (base_period) int64 2020
  * percentile   (percentile) object 'recommendation' 'p05-p50' ... 'p95-p50'
Data variables:
    Delta T      (percentile, scenario, variable, year, base_period) float64 0.0 ... nan

In [25]:
ds_DT.sel(year=2040, scenario='ssp119', percentile='recommendation', variable=v_sum)#.variables#.plot()#, variable='%%SVG')%%SVG

<xarray.Dataset>
Dimensions:      (base_period: 1)
Coordinates:
    year         int64 2040
    variable     <U41 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)'
    scenario     <U6 'ssp119'
  * base_period  (base_period) int64 2020
    percentile   <U14 'recommendation'
Data variables:
    Delta T      (base_period) float64 0.1408

## Make csv table of total values:

In [26]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

In [27]:
reference_year = ref_year
start_y_tabel = 2015
end_y_tabel = last_y

In [28]:
def get_fn(var_name, s_y, e_y, ref_y, perc):
    _st = var_name.replace('(','').replace(')','').replace(' ','_').replace(',','')#+'.csv'
    fn = f'{perc}_{_st}_{s_y}-{e_y}_refyear{ref_y}.csv'
    return fn

def make_sum_slcfs_tabel(sum_vn = vn_sum, percentile=recommendation):
    #_str = ''
    #_vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_erf_comp]
    #for var in _vl:
    #    _str += f'{var}, '
    #vn_sum = 'Sum SLCF (%s)' % _str[:-2]

    fn = get_fn(vn_sum, start_y_tabel, end_y_tabel, ref_year, percentile)



    # ref year value:
    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=sum_vn,
        year=reference_year
    ).squeeze()

    # all values from s_y to e_y
    _da = ds_DT[name_deltaT].sel(
        variable=sum_vn,
        year=slice(start_y_tabel, end_y_tabel)
    ) - ds_DT_sy

    # Choose recommendation::
    _pl_da = _da.sel(percentile=percentile).squeeze()
    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    fn = TABLE_DIR / fn
    print(percentile)
    display(df.loc[2040])
    df.to_csv(fn)
    
    
#for prc in [recommendation, 'p05-p50','p95-p50']:
#    make_sum_slcfs_tabel(percentile=prc)

In [29]:
import pandas as pd

## TABLE EACH VAR

In [30]:

def make_slcfs_tabel(var, percentile=recommendation):
    _str = ''
    _st = get_fn(var, start_y_tabel, end_y_tabel, ref_year, percentile)


    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=var, 
        year=reference_year,
    ).squeeze()
    
    _da = ds_DT[name_deltaT].sel(
        variable=var,
        year=slice(start_y_tabel, end_y_tabel)) - ds_DT_sy
    
    # Take recommendation::
    _pl_da = _da.sel(percentile=recommendation).squeeze()

    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    display(df)
    fn = TABLE_DIR/_st
    df.to_csv(fn)

for var in variables_erf_comp+[vn_sum]:
    for prc in [recommendation, 'p05-p50','p95-p50']:
        make_slcfs_tabel(var, percentile=prc)

scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.010401,-0.010133,-0.011205,-0.012933,-0.011700,-0.011700,-0.011489,-0.012286,-0.012286,-0.011137,-0.011011,recommendation
2016,-0.008543,-0.008274,-0.009347,-0.011075,-0.009842,-0.009842,-0.009630,-0.010428,-0.010428,-0.009279,-0.009152,recommendation
2017,-0.006706,-0.006437,-0.007510,-0.009238,-0.008004,-0.008004,-0.007793,-0.008591,-0.008591,-0.007441,-0.007315,recommendation
2018,-0.004815,-0.004547,-0.005619,-0.007347,-0.006114,-0.006114,-0.005903,-0.006700,-0.006700,-0.005551,-0.005425,recommendation
2019,-0.002883,-0.002615,-0.003687,-0.005415,-0.004182,-0.004182,-0.003971,-0.004769,-0.004769,-0.003619,-0.003493,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157695,-0.149583,0.000569,-0.006974,0.264628,0.264628,-0.124143,0.125497,0.178489,-0.161955,0.153379,recommendation
2097,-0.159087,-0.151086,0.000072,-0.006790,0.267536,0.267536,-0.123752,0.125400,0.178582,-0.163402,0.151810,recommendation
2098,-0.160446,-0.152544,-0.000392,-0.006600,0.270424,0.270424,-0.123330,0.125269,0.178660,-0.164771,0.150250,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.010401,-0.010133,-0.011205,-0.012933,-0.011700,-0.011700,-0.011489,-0.012286,-0.012286,-0.011137,-0.011011,p05-p50
2016,-0.008543,-0.008274,-0.009347,-0.011075,-0.009842,-0.009842,-0.009630,-0.010428,-0.010428,-0.009279,-0.009152,p05-p50
2017,-0.006706,-0.006437,-0.007510,-0.009238,-0.008004,-0.008004,-0.007793,-0.008591,-0.008591,-0.007441,-0.007315,p05-p50
2018,-0.004815,-0.004547,-0.005619,-0.007347,-0.006114,-0.006114,-0.005903,-0.006700,-0.006700,-0.005551,-0.005425,p05-p50
2019,-0.002883,-0.002615,-0.003687,-0.005415,-0.004182,-0.004182,-0.003971,-0.004769,-0.004769,-0.003619,-0.003493,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157695,-0.149583,0.000569,-0.006974,0.264628,0.264628,-0.124143,0.125497,0.178489,-0.161955,0.153379,p05-p50
2097,-0.159087,-0.151086,0.000072,-0.006790,0.267536,0.267536,-0.123752,0.125400,0.178582,-0.163402,0.151810,p05-p50
2098,-0.160446,-0.152544,-0.000392,-0.006600,0.270424,0.270424,-0.123330,0.125269,0.178660,-0.164771,0.150250,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.010401,-0.010133,-0.011205,-0.012933,-0.011700,-0.011700,-0.011489,-0.012286,-0.012286,-0.011137,-0.011011,p95-p50
2016,-0.008543,-0.008274,-0.009347,-0.011075,-0.009842,-0.009842,-0.009630,-0.010428,-0.010428,-0.009279,-0.009152,p95-p50
2017,-0.006706,-0.006437,-0.007510,-0.009238,-0.008004,-0.008004,-0.007793,-0.008591,-0.008591,-0.007441,-0.007315,p95-p50
2018,-0.004815,-0.004547,-0.005619,-0.007347,-0.006114,-0.006114,-0.005903,-0.006700,-0.006700,-0.005551,-0.005425,p95-p50
2019,-0.002883,-0.002615,-0.003687,-0.005415,-0.004182,-0.004182,-0.003971,-0.004769,-0.004769,-0.003619,-0.003493,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157695,-0.149583,0.000569,-0.006974,0.264628,0.264628,-0.124143,0.125497,0.178489,-0.161955,0.153379,p95-p50
2097,-0.159087,-0.151086,0.000072,-0.006790,0.267536,0.267536,-0.123752,0.125400,0.178582,-0.163402,0.151810,p95-p50
2098,-0.160446,-0.152544,-0.000392,-0.006600,0.270424,0.270424,-0.123330,0.125269,0.178660,-0.164771,0.150250,p95-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.061064,-0.060183,-0.057001,-0.023683,-0.039544,-0.046093,-0.046093,-0.049177,-0.049177,-0.065917,-0.065917,recommendation
2016,-0.045652,-0.044771,-0.041589,-0.008271,-0.024132,-0.030681,-0.030681,-0.033765,-0.033765,-0.050505,-0.050505,recommendation
2017,-0.031238,-0.030357,-0.027175,0.006143,-0.009718,-0.016267,-0.016267,-0.019351,-0.019351,-0.036091,-0.036091,recommendation
2018,-0.021071,-0.020190,-0.017008,0.016310,0.000448,-0.006100,-0.006100,-0.009184,-0.009184,-0.025924,-0.025924,recommendation
2019,-0.012831,-0.011950,-0.008768,0.024550,0.008688,0.002140,0.002140,-0.000944,-0.000944,-0.017684,-0.017684,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.413619,0.422858,0.273326,0.314578,-0.032630,0.277754,0.277754,0.246333,0.166807,0.358798,0.213283,recommendation
2097,0.416375,0.425736,0.275644,0.316485,-0.031047,0.279549,0.279549,0.249898,0.169828,0.359239,0.218014,recommendation
2098,0.419141,0.428593,0.277888,0.318385,-0.029431,0.281351,0.281351,0.253491,0.172808,0.359673,0.222796,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.061064,-0.060183,-0.057001,-0.023683,-0.039544,-0.046093,-0.046093,-0.049177,-0.049177,-0.065917,-0.065917,p05-p50
2016,-0.045652,-0.044771,-0.041589,-0.008271,-0.024132,-0.030681,-0.030681,-0.033765,-0.033765,-0.050505,-0.050505,p05-p50
2017,-0.031238,-0.030357,-0.027175,0.006143,-0.009718,-0.016267,-0.016267,-0.019351,-0.019351,-0.036091,-0.036091,p05-p50
2018,-0.021071,-0.020190,-0.017008,0.016310,0.000448,-0.006100,-0.006100,-0.009184,-0.009184,-0.025924,-0.025924,p05-p50
2019,-0.012831,-0.011950,-0.008768,0.024550,0.008688,0.002140,0.002140,-0.000944,-0.000944,-0.017684,-0.017684,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.413619,0.422858,0.273326,0.314578,-0.032630,0.277754,0.277754,0.246333,0.166807,0.358798,0.213283,p05-p50
2097,0.416375,0.425736,0.275644,0.316485,-0.031047,0.279549,0.279549,0.249898,0.169828,0.359239,0.218014,p05-p50
2098,0.419141,0.428593,0.277888,0.318385,-0.029431,0.281351,0.281351,0.253491,0.172808,0.359673,0.222796,p05-p50


...

scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.188779,-0.187112,-0.189428,-0.157321,-0.177850,-0.182137,-0.181797,-0.184046,-0.184047,-0.201465,-0.201091,p05-p50
2016,-0.146884,-0.145217,-0.147533,-0.115426,-0.135955,-0.140242,-0.139902,-0.142152,-0.142152,-0.159570,-0.159196,p05-p50
2017,-0.107127,-0.105459,-0.107776,-0.075669,-0.096198,-0.100485,-0.100145,-0.102394,-0.102394,-0.119813,-0.119439,p05-p50
2018,-0.070770,-0.069103,-0.071419,-0.039312,-0.059842,-0.064129,-0.063789,-0.066038,-0.066038,-0.083457,-0.083082,p05-p50
2019,-0.035834,-0.034167,-0.036483,-0.004376,-0.024906,-0.029193,-0.028853,-0.031102,-0.031102,-0.048520,-0.048146,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.139719,0.561359,1.716081,0.896218,2.964757,3.187480,2.484364,1.097904,2.194844,0.931462,3.903706,p05-p50
2097,0.133798,0.556645,1.725457,0.896927,3.010229,3.232684,2.524258,1.095867,2.211337,0.918999,3.955654,p05-p50
2098,0.127806,0.552061,1.734536,0.897603,3.055849,3.278011,2.564354,1.093386,2.227474,0.906663,4.007156,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.188779,-0.187112,-0.189428,-0.157321,-0.177850,-0.182137,-0.181797,-0.184046,-0.184047,-0.201465,-0.201091,p95-p50
2016,-0.146884,-0.145217,-0.147533,-0.115426,-0.135955,-0.140242,-0.139902,-0.142152,-0.142152,-0.159570,-0.159196,p95-p50
2017,-0.107127,-0.105459,-0.107776,-0.075669,-0.096198,-0.100485,-0.100145,-0.102394,-0.102394,-0.119813,-0.119439,p95-p50
2018,-0.070770,-0.069103,-0.071419,-0.039312,-0.059842,-0.064129,-0.063789,-0.066038,-0.066038,-0.083457,-0.083082,p95-p50
2019,-0.035834,-0.034167,-0.036483,-0.004376,-0.024906,-0.029193,-0.028853,-0.031102,-0.031102,-0.048520,-0.048146,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.139719,0.561359,1.716081,0.896218,2.964757,3.187480,2.484364,1.097904,2.194844,0.931462,3.903706,p95-p50
2097,0.133798,0.556645,1.725457,0.896927,3.010229,3.232684,2.524258,1.095867,2.211337,0.918999,3.955654,p95-p50
2098,0.127806,0.552061,1.734536,0.897603,3.055849,3.278011,2.564354,1.093386,2.227474,0.906663,4.007156,p95-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.079461,-0.077896,-0.078579,-0.042700,-0.065697,-0.069982,-0.069626,-0.073948,-0.073948,-0.089422,-0.089218,recommendation
2016,-0.059926,-0.058361,-0.059044,-0.023165,-0.046161,-0.050447,-0.050091,-0.054412,-0.054412,-0.069887,-0.069683,recommendation
2017,-0.042070,-0.040505,-0.041189,-0.005310,-0.028306,-0.032592,-0.032236,-0.036557,-0.036557,-0.052032,-0.051827,recommendation
2018,-0.027449,-0.025884,-0.026567,0.009312,-0.013685,-0.017970,-0.017614,-0.021936,-0.021936,-0.037410,-0.037206,recommendation
2019,-0.013987,-0.012422,-0.013106,0.022773,-0.000223,-0.004509,-0.004153,-0.008474,-0.008474,-0.023949,-0.023744,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.105075,0.127615,0.287193,0.236441,0.550174,0.782678,0.134256,0.414923,0.485560,0.081898,0.646143,recommendation
2097,0.106044,0.128203,0.288523,0.238634,0.558343,0.790873,0.137515,0.417453,0.487155,0.080115,0.647954,recommendation
2098,0.107090,0.128854,0.289839,0.240843,0.566526,0.799056,0.140837,0.419978,0.488728,0.078475,0.649735,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.079461,-0.077896,-0.078579,-0.042700,-0.065697,-0.069982,-0.069626,-0.073948,-0.073948,-0.089422,-0.089218,p05-p50
2016,-0.059926,-0.058361,-0.059044,-0.023165,-0.046161,-0.050447,-0.050091,-0.054412,-0.054412,-0.069887,-0.069683,p05-p50
2017,-0.042070,-0.040505,-0.041189,-0.005310,-0.028306,-0.032592,-0.032236,-0.036557,-0.036557,-0.052032,-0.051827,p05-p50
2018,-0.027449,-0.025884,-0.026567,0.009312,-0.013685,-0.017970,-0.017614,-0.021936,-0.021936,-0.037410,-0.037206,p05-p50
2019,-0.013987,-0.012422,-0.013106,0.022773,-0.000223,-0.004509,-0.004153,-0.008474,-0.008474,-0.023949,-0.023744,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.105075,0.127615,0.287193,0.236441,0.550174,0.782678,0.134256,0.414923,0.485560,0.081898,0.646143,p05-p50
2097,0.106044,0.128203,0.288523,0.238634,0.558343,0.790873,0.137515,0.417453,0.487155,0.080115,0.647954,p05-p50
2098,0.107090,0.128854,0.289839,0.240843,0.566526,0.799056,0.140837,0.419978,0.488728,0.078475,0.649735,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.079461,-0.077896,-0.078579,-0.042700,-0.065697,-0.069982,-0.069626,-0.073948,-0.073948,-0.089422,-0.089218,p95-p50
2016,-0.059926,-0.058361,-0.059044,-0.023165,-0.046161,-0.050447,-0.050091,-0.054412,-0.054412,-0.069887,-0.069683,p95-p50
2017,-0.042070,-0.040505,-0.041189,-0.005310,-0.028306,-0.032592,-0.032236,-0.036557,-0.036557,-0.052032,-0.051827,p95-p50
2018,-0.027449,-0.025884,-0.026567,0.009312,-0.013685,-0.017970,-0.017614,-0.021936,-0.021936,-0.037410,-0.037206,p95-p50
2019,-0.013987,-0.012422,-0.013106,0.022773,-0.000223,-0.004509,-0.004153,-0.008474,-0.008474,-0.023949,-0.023744,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.105075,0.127615,0.287193,0.236441,0.550174,0.782678,0.134256,0.414923,0.485560,0.081898,0.646143,p95-p50
2097,0.106044,0.128203,0.288523,0.238634,0.558343,0.790873,0.137515,0.417453,0.487155,0.080115,0.647954,p95-p50
2098,0.107090,0.128854,0.289839,0.240843,0.566526,0.799056,0.140837,0.419978,0.488728,0.078475,0.649735,p95-p50
